In [27]:
import pandas as pd 

In [28]:
import numpy as np

In [29]:
!ls ../out

output_CM_global.csv	output_SLATM_global.csv
output_FCHL_global.csv	output_SOAP_global.csv


In [30]:
slatm_out = pd.read_csv("../out/output_SOAP_global.csv", index_col=0)

In [31]:
slatm_out

,SolN,Fragments,ObjValNoPen,ObjValWithPen
0,1,"['frag_06_c00001', 'frag_14_c00001']",0.043947,0.043947
1,2,"['frag_08_c00001', 'frag_12_c00001']",0.046057,0.046057
2,3,"['frag_08_c00001', 'frag_11_c00001']",0.046654,0.046654
3,4,"['frag_10_c00001', 'frag_12_c00001']",0.047774,0.047774
4,5,"['frag_06_c00001', 'frag_13_c00001']",0.048499,0.048499
...,...,...,...,...
995,996,"['frag_04_c00001', 'frag_06_c00001', 'frag_11_...",0.433316,0.503316
996,997,"['frag_01_c00001', 'frag_04_c00001', 'frag_06_...",0.493346,0.503346
997,998,"['frag_06_c00001', 'frag_10_c00001', 'frag_11_...",0.403365,0.503365
998,999,"['frag_03_c00001', 'frag_05_c00001', 'frag_09_...",0.463494,0.503494


In [32]:
from ast import literal_eval 

In [33]:
slatm_out["Fragments"] = slatm_out["Fragments"].apply(literal_eval)

In [34]:
# excess is missing Hs - to fix 

In [35]:
import qml

In [36]:
qm9_mol = qml.Compound("../targets/qm9_0.xyz")
qm9_ncharges = qm9_mol.nuclear_charges

In [37]:
qm9_ncharges

array([8, 6, 6, 7, 6, 8, 8, 7, 6, 1, 1])

In [38]:
def count_ncharges(list_ncharges, elements=[1,6,7,8,16]):
    ncharge_count = np.zeros(len(elements))
    for ncharge in list_ncharges:
        for i, elem in enumerate(elements):
            if ncharge == elem:
                ncharge_count[i] += 1
    return ncharge_count

In [39]:
qm9_counts = count_ncharges(qm9_ncharges)

In [40]:
qm9_counts

array([2., 4., 2., 3., 0.])

In [44]:
def get_excess_frags(frag_list):
    mols = [qml.Compound("../amons-qm9/"+x+".xyz") for x in frag_list]
    ncharges = np.concatenate([mol.nuclear_charges for mol in mols])
    ncharge_count = count_ncharges(ncharges)
    
    diff_count = ncharge_count - qm9_counts
    diff_count = np.array(diff_count).astype(int)
    excess = []
    for i, count in enumerate(diff_count): 
        ncharge = [1,6,7,8,16][i]
        if count > 0 :
            for i in range(count):
                excess.append(ncharge)
        elif count < 0:
            for i in range(abs(count)):
                excess.append(-ncharge)
    return excess

In [45]:
slatm_out["Excess"] = slatm_out["Fragments"].apply(get_excess_frags)

In [46]:
slatm_out.sort_values(by='ObjValWithPen').iloc[0:50]

,SolN,Fragments,ObjValNoPen,ObjValWithPen,Excess
0,1,"[frag_06_c00001, frag_14_c00001]",0.043947,0.043947,"[1, 1, 1, 1]"
1,2,"[frag_08_c00001, frag_12_c00001]",0.046057,0.046057,"[1, 1, 1, 1]"
2,3,"[frag_08_c00001, frag_11_c00001]",0.046654,0.046654,"[1, 1, 1, 1]"
3,4,"[frag_10_c00001, frag_12_c00001]",0.047774,0.047774,"[1, 1, 1, 1]"
4,5,"[frag_06_c00001, frag_13_c00001]",0.048499,0.048499,"[1, 1, 1, 1, 1, 7, -8]"
5,6,"[frag_10_c00001, frag_11_c00001]",0.050336,0.050336,"[1, 1, 1, 1]"
6,7,"[frag_05_c00001, frag_13_c00001]",0.053927,0.053927,"[1, 1, 1, 1]"
7,8,"[frag_03_c00001, frag_15_c00001]",0.054130,0.054130,"[1, 1, 1, 1]"
8,9,"[frag_06_c00001, frag_15_c00001]",0.044922,0.054922,"[1, 1, 1, 1, 8]"
9,10,"[frag_06_c00001, frag_16_c00001]",0.045124,0.055124,"[1, 1, 1, 7]"
